In [ ]:
# Setup the environment and import necessary libraries
from google.cloud import bigquery
import pandas as pd

# Initialize the BigQuery client
client = bigquery.Client(project="project-dsai-2025-11-12")

df_orders = client.query("""
SELECT *
FROM `project-dsai-2025-11-12.olist_stage.fct_orders`
""").to_dataframe()

df_orders.head()

/home/taufiqidris/miniconda3/envs/elt/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,hrs_to_approve,days_to_deliver,days_early_late_vs_estimated
0,bd35b677fd239386e9861d11ae98ab56,ad2eb5d02c0808bcd8743117f6383cf6,canceled,2018-09-17 17:21:16+00:00,NaT,NaT,NaT,2018-10-01 00:00:00+00:00,<NA>,<NA>,<NA>
1,5f787bb3e0a01a4f347192157c036791,61954273c451808b32e15fb11dbf9116,canceled,2017-03-13 06:03:37+00:00,2017-03-13 06:03:37+00:00,NaT,NaT,2017-04-03 00:00:00+00:00,0,<NA>,<NA>
2,2aaab7e991347226dbda1f61c6785f88,55c72a24117236fa6e1e89bb2b4a02e0,canceled,2018-07-25 13:57:52+00:00,2018-07-25 15:10:12+00:00,NaT,NaT,2018-08-10 00:00:00+00:00,1,<NA>,<NA>
3,84002f994c72707b99788fbbc6f72f3f,da46ea1fd34482beef6ea70f81442d75,canceled,2018-06-29 09:20:04+00:00,2018-06-29 09:35:14+00:00,NaT,NaT,2018-07-23 00:00:00+00:00,0,<NA>,<NA>
4,24b425b6d0f061fcb84c1832d221e3bb,aeb4c7533882b7b165d846a5cb5a4c3e,canceled,2017-08-22 09:50:59+00:00,NaT,NaT,NaT,2017-09-26 00:00:00+00:00,<NA>,<NA>,<NA>


In [ ]:
# Check row counts for all tables in the staging area
tables = [
    "fct_orders",
    "fct_order_items",
    "fct_payments",
    "fct_reviews",
    "dim_customers",
    "dim_products",
    "dim_sellers",
    "dim_geolocation"
]

counts = []

for table in tables:
    q = f"""
    SELECT COUNT(*) AS row_count
    FROM `project-dsai-2025-11-12.olist_stage.{table}`
    """
    row_count = client.query(q).to_dataframe().iloc[0, 0]
    counts.append({"table": table, "rows": row_count})

#pd.DataFrame(counts)

/home/taufiqidris/miniconda3/envs/elt/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,table,rows
0,fct_orders,99439
1,fct_order_items,112642
2,fct_payments,99437
3,fct_reviews,98407
4,dim_customers,99440
5,dim_products,32950
6,dim_sellers,3094
7,dim_geolocation,19014


In [ ]:
# Primary key checks for all tables
pk_checks = {
    "dim_customers": "customer_id",
    "dim_products": "product_id",
    "dim_sellers": "seller_id",
    "fct_orders": "order_id",
    "fct_reviews": "review_id"
}

results = []

for table, pk in pk_checks.items():
    q = f"""
    SELECT
      COUNT(*) AS total_rows,
      COUNT(DISTINCT {pk}) AS distinct_pk
    FROM `project-dsai-2025-11-12.olist_stage.{table}`
    """
    df = client.query(q).to_dataframe()
    results.append({
        "table": table,
        "pk": pk,
        "total_rows": df.total_rows[0],
        "distinct_pk": df.distinct_pk[0]
    })

pd.DataFrame(results)

/home/taufiqidris/miniconda3/envs/elt/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,table,pk,total_rows,distinct_pk
0,dim_customers,customer_id,99440,99440
1,dim_products,product_id,32950,32950
2,dim_sellers,seller_id,3094,3094
3,fct_orders,order_id,99439,99439
4,fct_reviews,review_id,98407,98407


In [ ]:
# Null checks for critical columns in fact tables
null_checks = {
    "fct_orders": ["order_id", "customer_id", "order_purchase_timestamp"],
    "fct_order_items": ["order_id", "product_id", "seller_id", "price"],
    "fct_payments": ["order_id", "payment_value"],
    "fct_reviews": ["order_id", "review_score"]
}

null_results = []

for table, cols in null_checks.items():
    for col in cols:
        q = f"""
        SELECT COUNT(*) AS null_count
        FROM `project-dsai-2025-11-12.olist_stage.{table}`
        WHERE {col} IS NULL
        """
        nulls = client.query(q).to_dataframe().iloc[0, 0]
        null_results.append({
            "table": table,
            "column": col,
            "null_count": nulls
        })

pd.DataFrame(null_results).query("null_count > 0")

/home/taufiqidris/miniconda3/envs/elt/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,table,column,null_count


In [ ]:
# Date range checks
q = """
SELECT
  MIN(order_purchase_timestamp) AS min_date,
  MAX(order_purchase_timestamp) AS max_date
FROM `project-dsai-2025-11-12.olist_stage.fct_orders`
"""
client.query(q).to_dataframe()

/home/taufiqidris/miniconda3/envs/elt/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,min_date,max_date
0,2016-09-04 21:15:19+00:00,2018-10-17 17:30:18+00:00


In [ ]:
# Temporal consistency checks
q = """
SELECT
  COUNT(*) AS bad_dates
FROM `project-dsai-2025-11-12.olist_stage.fct_orders`
WHERE order_delivered_customer_date < order_purchase_timestamp
"""
client.query(q).to_dataframe()

/home/taufiqidris/miniconda3/envs/elt/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,bad_dates
0,0


In [ ]:
# Referential integrity checks - orphan orders without customers
q = """
SELECT COUNT(*) AS orphan_orders
FROM `project-dsai-2025-11-12.olist_stage.fct_orders` o
LEFT JOIN `project-dsai-2025-11-12.olist_stage.dim_customers` c
  ON o.customer_id = c.customer_id
WHERE c.customer_id IS NULL
"""
client.query(q).to_dataframe()

/home/taufiqidris/miniconda3/envs/elt/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,orphan_orders
0,0


In [ ]:
# Price range checks
q = """
SELECT
  MIN(price) AS min_price,
  MAX(price) AS max_price,
  AVG(price) AS avg_price
FROM `project-dsai-2025-11-12.olist_stage.fct_order_items`
"""
client.query(q).to_dataframe()

/home/taufiqidris/miniconda3/envs/elt/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,min_price,max_price,avg_price
0,0.850000000,6735.000000000,120.656604197


In [ ]:
# Review score checks
q = """
SELECT
  MIN(review_score) AS min_score,
  MAX(review_score) AS max_score
FROM `project-dsai-2025-11-12.olist_stage.fct_reviews`
"""
client.query(q).to_dataframe()

/home/taufiqidris/miniconda3/envs/elt/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,min_score,max_score
0,1,5


### Data Validation Summary

- All fact and dimension tables populated
- Primary keys unique across dimensions and facts
- No critical nulls detected in IDs or metrics
- Date ranges and delivery timelines consistent
- Referential integrity validated via joins
- Metrics within expected business ranges

Dataset is analysis-ready.